In [1]:
from __future__ import division
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import model_from_json
import numpy as np
import random

Using TensorFlow backend.


In [2]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    if sum(a) > 1.0:  # occasionally getting 1.00000X, so handling for that
        a *= .999
    return np.argmax(np.random.multinomial(1, a, 1))

In [9]:
def train():
    """trains the LSTM model on text corpora"""

    path = "ADVENTURES_OF_SHERLOCK_HOLMES.txt"

    try:
        text = open(path).read().lower()
    except UnicodeDecodeError:
        import codecs
        text = codecs.open(path, encoding='utf-8').read().lower()
        
    print('corpus length:', len(text))
    
    text = text[:100777]

    chars = set(text)
    words = set(text.split())

    print("total number of unique words", len(words))
    print("total number of unique chars", len(chars))

    word_indices = dict((c, i) for i, c in enumerate(words))
    indices_word = dict((i, c) for i, c in enumerate(words))

    maxlen = 30
    step = 3

    print("maxlen:", maxlen,"step:", step)

    sentences = []
    next_words = []
    next_words = []
    list_words = []

    sentences2 = []
    list_words = text.lower().split()


    for i in range(0, len(list_words) - maxlen, step):
        sentences2 = ' '.join(list_words[i: i + maxlen])
        sentences.append(sentences2)
        next_words.append((list_words[i + maxlen]))

    print('length of sentence list:', len(sentences))
    print("length of next_word list", len(next_words))

    print('Vectorization...')
    X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
    y = np.zeros((len(sentences), len(words)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, word in enumerate(sentence.split()):
            X[i, t, word_indices[word]] = 1
        y[i, word_indices[next_words[i]]] = 1


    #build the model: 2 stacked LSTM

    print('Building model...')

    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(words))))
    model.add(Dropout(0.2))
    model.add(LSTM(512, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(len(words)))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    try:
        model.load_weights("temp_weights.txt")
    except Exception as e:
        print(e)
        pass

    # train the model, output generated text after each iteration

    for iteration in range(1, 25):
        print('-' * 50)
        print('Iteration', iteration)

        model.fit(X, y, batch_size=500, nb_epoch=3)
        json_string = model.to_json()
        with open('<path to where you want to save your model architecture>', 'w') as f:
            f.write(json_string)
        model.save_weights('temp_weights.txt', overwrite=True)


In [10]:
def generate_from_word_level_rnn(maxlen=30, diversity=1.0, min_sent_len=10, max_sent_len=65):
    with open("ADVENTURES_OF_SHERLOCK_HOLMES.txt", "r") as f:
        text = f.read().lower().split()[:4940]
    words = set(text)
    start_index = random.randint(0, len(text) - maxlen - 1)
    word_indices = dict((c, i) for i, c in enumerate(words))
    indices_word = dict((i, c) for i, c in enumerate(words))

    response = ""
    model = model_from_json(open('ADVENTURES_OF_SHERLOCK_HOLMES.txt').read())
    model.load_weights('temp_weights.txt')
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    sentence = text[start_index: start_index + maxlen]

    for i in range(random.randint(min_sent_len, max_sent_len)):
        x = np.zeros((1, maxlen, len(words)))
        for t, word in enumerate(sentence):
            x[0, t, word_indices[word]] = 1.
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_word = indices_word[next_index]
        if not response:
            response += ' {0}'.format(next_word)
        else:
            if response.split()[-1] != next_word:
                response += ' {0}'.format(next_word)
        del sentence[0]
        sentence.append(next_word)
    return response


    ### to load this model elsewhere... ###

    # from keras.models import model_from_json

    # model = model_from_json(open('<path to your saved model architecture>').read())
    # model.load_weights('<path to your saved weights>')
    # model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
train()
generate_from_word_level_rnn()

corpus length: 592333
total number of unique words 4551
total number of unique chars 59
maxlen: 30 step: 3
length of sentence list: 6151
length of next_word list 6151
Vectorization...
Building model...
Dimension 0 in both shapes must be equal, but are 4551 and 475. Shapes are [4551,2048] and [475,2048]. for 'Assign_16' (op: 'Assign') with input shapes: [4551,2048], [475,2048].
--------------------------------------------------
Iteration 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/3
6151/6151 [==============================] - 167s 27ms/step - loss: 7.2659
Epoch 2/3
6151/6151 [==============================] - 166s 27ms/step - loss: 6.5259
Epoch 3/3
6151/6151 [==============================] - 48357s 8s/step - loss: 6.4557
--------------------------------------------------
Iteration 2
Epoch 1/3
6151/6151 [==============================] - 187s 30ms/step - loss: 6.4229
Epoch 2/3
6151/6151 [==============================] - 180s 29ms/step - loss: 6.3993
Epoch 3/3
6151/6151 [==============================] - 179s 29ms/step - loss: 6.3941
--------------------------------------------------
Iteration 3
Epoch 1/3
6151/6151 [==============================] - 87381s 14s/step - loss: 6.3787
Epoch 2/3
6151/6151 [==============================] - 181s 29ms/step - loss: 6.3646
Epoch 3/3
6151/6151 [==============================] - 398s 65ms/step - loss: 6.3440
--------------------------------------------------
Iteration 4
Epoch 1/3
3000/6151 [=============>..........